<a href="https://colab.research.google.com/github/ChoiSenn/Study/blob/main/AI/KoBERT/KoBERT_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98_%ED%95%A9%EC%9D%BC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7hbkzhqw
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7hbkzhqw
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [36]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [37]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [38]:
#GPU 사용
device = torch.device("cuda:0")

In [39]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [40]:
import csv
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/AI/dataset/train.hate.csv',encoding="utf-8")
train2 = pd.read_csv('/content/drive/MyDrive/AI/dataset/train_curse.csv',encoding="utf-8")

In [41]:
train.sample(n=10)

,comments,label
6087,"인맥자랑, 보여주기식 그이상 그이하도 아니다",offensive
958,그와중에 조보아 너무 예쁘다....ㅠ,none
6798,좌빨 영화 납시요 개돼지들 선동시키기 딱이요,hate
5829,이병헌이 뭔 잘못했어요?저는 내마음의 풍금 봤는데 잘생기게 나와서 좋더만,none
5820,이발은 다른사람이 하고 저분들은 뭐함?,none
2767,민폐남주 내가 어떻게 행동해도 만월이 도울꺼니 괜찮아라니.. 그리고 까칠한 사장 직...,none
2000,댓글 성비랑 나이대보면 소름이 돋는다. 30~40대 아지매들 밥쳐먹고 할일없어 연예...,offensive
772,그냥 둘이 맘에있는듯~~홍진영은 등치있고 리더십있는 상남자스탈을 좋아하고 김종국도 ...,none
879,그래서 이 와중에도 박한별을 욕한다는게ㅋㅋㅋㅋㅋㅋ역시 대단한 나라야,hate
810,그냥적당히하시지 나도40대지만 솔까 반짝했지 글구옛날노래다시부르는거 반칙아님 다른가...,none


In [42]:
train2.sample(n=10)

,comments,label
2754,씹벌레색기가 머라는거야..? ㅋㅋㅋ,1
5253,금리 10프로 옛말이다 .. 거기서 3년전까지 일했는데 그때도 7퍼 되나마나 했는데,0
4530,자기 주제를 알면 1번은 쳐다보지 않지 2 3번 정도가 맘에드는 사람도 있다 물론 ...,0
2327,우리나라 저능아들은 주적이 미국이라든데 ㅋㅋㅋㅋㅋㅋ,1
1673,카라반양반 참... 생각이 있으면 후미진데로 옮겨야지 참...답답하네. 차가 3대인...,0
465,저열한 민족성 미국 이승만 박정희라는 변수가 있어서 본래 민족 모습을 잠시 ...,1
653,철학=종교,0
2131,넌 같은 지능의 사냥꾼 머리랑 회계사 머리가 같다고 보냐?,1
4437,자성회(自省會) 또는 자생단은 1919년 4월 6일부터 12월까지 활동한 일제 강점...,0
2275,할게 없으니까ㅋㅋ,0


In [43]:
train.loc[(train['label'] == "none"), 'label'] = 0
train.loc[(train['label'] == "hate"), 'label'] = 1
train.loc[(train['label'] == "offensive"), 'label'] = 1

In [44]:
train = pd.concat([train, train2])
print(len(train))
train.sample(n=10)

13721


,comments,label
2147,눈깔이 사신가;;,1
6889,지금까지 해온것처럼 씩씩하고 밝은 준희양으로 자라줬으면 좋겠어. 항상 응원할께~~,0
1047,앱등이새끼들 아닥하노?,1
2316,강남은 무료임 ㅎㅎ,0
834,한국은 좆같은 황사가 날아오지만역살적으로 땅의 회복력이 올라감,1
2101,꽈자와자꽈꿍꽐라까자 '쾈' 자불라자,0
2805,오..,0
1929,"일본은 중견기업, 대기업이 한국보다 훨씬 많아서한국은 586이 은퇴해도 저렇게 될거...",0
4874,연기는 다 잘하네 이병헌 유연석 김민정 변요한 통역관은 잘하는데 김태리는 글쎄.. ...,0
6627,정준이 연예인이냐? 월세 못낸다고 징징대고 십년안에 연기한 작품이 잇기는 함? 쩌리...,1


In [45]:
data_list = []
for q, label in zip(train['comments'], train['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [46]:
print(data_list[0])
print(data_list[1000])
print(data_list[2000])
print(data_list[3000])
print(data_list[4000])
print(data_list[5000])
print(data_list[-1])

['(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....', '1']
['근데 솔직히 김정훈 천재유전자 받고 싶어하는 여자들 많을텐데..', '1']
['댓글 성비랑 나이대보면 소름이 돋는다. 30~40대 아지매들 밥쳐먹고 할일없어 연예인한테 악플다는게 취미인가 보네.', '1']
['범죄를 범죄인줄 모르고 ㅈㄹ떠는놈들이나 남자들 무데기로 있는 호텔방가서 ㅊ먹는 ㄴ이나 에혀저런 ㄴ들 만나결혼하면 인생 망친다', '1']
['아니ᆢᆢ지안이 황금빛내인생은 언제부터야ᆢᆢ', '0']
['예쁘다. 이혼녀라도 환영한다', '0']
['우리나라교회는 악마들이모여 주뎅이 처벌리고 ', '1']


In [47]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.20, random_state=0)

In [48]:
print(len(dataset_train))
print(len(dataset_test))

10976
2745


In [49]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [50]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [51]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [52]:
data_train[0]

(array([   2, 1394, 7141, 5702, 2351, 6733, 6797, 1198, 5771, 4586, 6821,
        6629, 6882, 6840, 2778, 7096, 4497, 1761, 7141, 7083, 2095, 7003,
        7318, 6241, 7102,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(26, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [53]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [54]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [55]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [56]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-56-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/172 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7672051787376404 train acc 0.4375
epoch 1 train acc 0.6370821220930233


<ipython-input-56-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/43 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7826652386780907


  0%|          | 0/172 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.23215259611606598 train acc 0.9375
epoch 5 train acc 0.9579396802325582


  0%|          | 0/43 [00:00<?, ?it/s]

epoch 5 test acc 0.7797582619339046


In [57]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("혐오가 안")
            elif np.argmax(logits) == 1:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 게이새끼들
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 동성애자들 물러가라
>> 입력하신 내용에서 혐오가 안 느껴집니다.


하고싶은 말을 입력해주세요 : 장애인 시위 못봐주겠네
>> 입력하신 내용에서 혐오가 안 느껴집니다.


하고싶은 말을 입력해주세요 : 장애인들 꼴보기 싫다
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 잼민이들이 꼭 나대더라
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 늙은이들 집 밖에 못 나오게 해라
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 집 밖에 못 나오게 해라
>> 입력하신 내용에서 혐오가 안 느껴집니다.


하고싶은 말을 입력해주세요 : 걔네가 꼭 나대더라
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 꼴보기 싫다
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 0
